# Tutorial: Introducción a Pyspark

## 1. Introducción
    ¿Qué aprenderá? 
    En este tutorial aprenderá sobre pyspark, dataframes, y persistencia

    ¿Qué construirá? 
    En este tutorial se familiarizará con el uso del paquete Pyspark
    
    ¿Para qué?
    Antes de realizar cualquier tipo de análisis, necesita conocimientos básicos sobre algunas tecnologías a usar a los largo del curso
    
    ¿Qué necesita?
    Los siguientes requisitos se encuentran instalados en la máquina virtual asignada a cada estudiante, específicamente en el ambiente de anaconda llamado "tutoriales". Recuerde que tiene a su disposición el tutorial de conexión a máquinas virtuales en la semana 1 de Coursera
    1. Python 3 con pip instalado
    2. Jupyter notebook
    3. Paquetes: Pyspark (3.0.1), pandas (1.2.1), numpy (1.20.0) y matplotlib (3.3.4)
    Otros:
    1. Controlador Connector J MySQL (ya se encuentra configurado)
    2. Acceso a servidor remoto MySQL con base de datos relacional "WWImportersTransactional". Recuerde que tiene a su disposición el tutorial de conexión remoto a Mysql en la semana 1 de Coursera

Para una contextualizacion general de pyspark dirijase al video tutorial <i>Sobre pyspark</i>

La documentación de Pyspark es pública y la encuentra en: 
https://spark.apache.org/docs/latest/api/python/index.html

Le recomendamos revisar la documentación y practicar dado que en los tutoriales no se cubriran todos los casos posibles de procesamiento de datos con Pyspark

## 2. Configuración e importe de paquetes
Se importan los paquetes de python necesarios y se configura el controlador de conexion entre pyspark y mysql

In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions
from pyspark.sql.types import StructType, StructField
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql.types import *
from pyspark.sql.types import FloatType, StringType, IntegerType, DateType
from pyspark.sql.functions import udf
import datetime
from datetime import datetime
import os 

El siguiente paso es una configuración necesaria para que el control del connector J de MySQL

In [2]:
path_jar_driver = 'C:\Program Files (x86)\MySQL\Connector J 8.0\mysql-connector-java-8.0.28.jar'

## 3. Crear una sesión de PySpark 

Primero, aprenderá a crear una sesión de Spark. Las sesiones de Spark son el punto principal de entrada para programar Spark con Dataframes (estructuras de datos en forma de tabla), como se hará en la siguiente sección, pues presenta el API con el que se pueden manipular estos objetos. Para crear una sesión básica, basta con ejecutar el código a continuación:  

``` 
spark = SparkSession.builder.appName('tutorial pyspark').getOrCreate()

La sesión creada arriba tiene como nombre <i> tutorial pyspark </i>, está ejecutándose en la máquina local y está utilizando las configuraciones por defecto de PySpark. Para Coursera Labs NO se incluye esta configuración

Aunque no será necesario para este curso, estas son algunas de las otras configuraciones que se pueden utilizar: 



 - Usando la función <i>master</i> se puede configurar el clúster computacional sobre el que se ejecutará. Por ejemplo, si tuviera un clúster en la dirección 192.168.1.3, podría escoger ejecutar esta función de PySpark en ese clúster usando la función así:  
 
 ``` 
 spark = SparkSession.builder.appName('tutorial pyspark').master('192.168.1.3').getOrCreate()
 
- La función <i>enableHiveSupport</i> habilita el uso de Spark junto con Apache Hive y habilita las funcionalides de Hive:
  ``` 
  spark = SparkSession.builder.appName('tutorial pyspark').master('192.168.1.3').enableHiveSupport().getOrCreate()

- Finalmente, la función <i>config</i> permite configurar opciones más avanzadas, si así se desea:
  ``` 
  spark = SparkSession.builder.appName('tutorial pyspark').master('192.168.1.3').enableHiveSupport().("spark.some.config.option", "some-value").getOrCreate()
  
Recuerde que solo se puede tener una sesion de Spark activa al mismo tiempo, si ejecute varias veces la siguiente celda le saldra un error en relación a este hecho

In [3]:
#Configuración de la sesión
conf=SparkConf() \
    .set('spark.driver.extraClassPath', path_jar_driver)

spark_context = SparkContext(conf=conf)
sql_context = SQLContext(spark_context)
spark = sql_context.sparkSession

C:\ProgramData\Anaconda3\envs\Tutoriales\lib\site-packages\pyspark\sql\context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


Para cerrar la sesión de Spark ejecute el comando <i>spark.stop()</i>. Recuerde que cualquier código que use la sesión spark no va a funcionar después de ejecutar el comando anterior

In [4]:
#spark.stop()

## 4. Dataframes

Un Dataframe es una estructura de datos de PySpark que permite trabajar con datos estructurados. Un Dataframe organiza los datos en una tabla compuesta por columnas, de manera similar como se estructura una tabla SQL.

Trabajar con un Dataframe permite utilizar el procesamiento distribuido de Spark de manera sencilla, sin tener que lidiar con las estructuras RDD de Spark, que son un poco más complicadas. 

Existen varias formas de crear un Dataframe, en este tutorial se verán tres:

1. Crear un Dataframe programáticamente.
2. Crear un Dataframe a partir de un archivo CSV.
3. Crear un Dataframe a partir de una tabla de una base de datos relacional.

La manipulación del dataframe en la etapa de preprocesamiento es independiente de la fuente de datos utilizada.

### 4.1. Dataframe programáticamente
Crear un Dataframe programáticamente es simple, pues basta con utilizar el método <i>createDataFrame</i> de la sesión de Spark. El método puede inferir el esquema del Dataframe o se le puede especificar el esquema.

En este ejemplo, especificamos un esquema con dos columnas <i>numeroEscrito</i>, de tipo <i>string</i>, y <i>numero</i>, de tipo <i>integer</i>.

In [5]:
esquema = StructType().add('ID_Cliente',IntegerType()).add('Nombre','string').add('ClienteFactura',IntegerType()).add('ID_Categoria',IntegerType()).add('ID_GrupoCompra',IntegerType()).add('ID_CiudadEntrega',IntegerType()).add('LimiteCredito',IntegerType()).add('FechaAperturaCuenta','date').add('DiasPago',IntegerType())
   
df_prog = spark.createDataFrame([(1,'Tailspin Toys (Head Office)',1,3,1,19586,0,datetime.strptime('2013-10-12','%Y-%m-%d'),7)], schema = esquema)

#la función .show() del Dataframe, imprime las primeas 20 filas del mismo.
df_prog.show()


+----------+--------------------+--------------+------------+--------------+----------------+-------------+-------------------+--------+
|ID_Cliente|              Nombre|ClienteFactura|ID_Categoria|ID_GrupoCompra|ID_CiudadEntrega|LimiteCredito|FechaAperturaCuenta|DiasPago|
+----------+--------------------+--------------+------------+--------------+----------------+-------------+-------------------+--------+
|         1|Tailspin Toys (He...|             1|           3|             1|           19586|            0|         2013-10-12|       7|
+----------+--------------------+--------------+------------+--------------+----------------+-------------+-------------------+--------+



### 4.2. Dataframe desde csv
Para crear un Dataframe a partir de un archivo CSV se puede utilizar el método <i>load</i> del <i>DataFrameReader</i> de la sesión. A este método se le puede especificar la ruta del archivo (se recomienda usar variables ej. <i>PATH</i>), el tipo de archivo, el separador, si se desea que infiera el esquema o se le proveerá un esquema y si el archivo contiene encabezado o no. A continuación, se muestra un ejemplo:

In [6]:
PATH = './'

In [7]:
df_csv = spark.read.load(PATH+"Clientes.csv", format="csv", sep=";", inferSchema="true")
df_csv.show()

+----------+--------------------+----------------+------------------+-------------+--------------+-----------+-----------------+-----------+
|       _c0|                 _c1|             _c2|               _c3|          _c4|           _c5|        _c6|              _c7|        _c8|
+----------+--------------------+----------------+------------------+-------------+--------------+-----------+-----------------+-----------+
|CustomerID|        CustomerName|BillToCustomerID|CustomerCategoryID|BuyingGroupID|DeliveryCityID|CreditLimit|AccountOpenedDate|PaymentDays|
|         1|Tailspin Toys (He...|               1|                 3|          1.0|         19586|       null|        1/01/2013|          7|
|         2|Tailspin Toys (Sy...|               1|                 3|          1.0|         33475|       null|        1/01/2013|          7|
|         3|Tailspin Toys (Pe...|               1|                 3|          1.0|         26483|       null|        1/01/2013|          7|
|         4|T

### 4.3. Carga de datos desde la base de datos relacional
Cargar datos de una base de datos relacional es un poco más difícil, pues hay que asegurarse de tener el driver adecuado para la conexión, la dirección IP, usuario, contraseña y tabla de la base de datos. (se recomienda usar variables Ej. db_user, db_psswd, db_connection_string)

Las bases de datos WWI_DWH_i hacen referencia a las de cada estudiante Estudiante_i, lo mismo sucede con las del proyecto Proyecto_DWH_I son Proyegto_Gi.

![Infraestructura](./Arquitectura_infraestructura.png)

Ahora, se cargará la tabla de clientes de Wide World Importers:

In [8]:
db_user = ''
db_psswd = ''
db_connection_string = 'jdbc:mysql://157.253.236.116:8080/WWImportersTransactional'

In [9]:
df_bd = spark.read.format('jdbc')\
    .option('url', db_connection_string) \
    .option('dbtable', '''(SELECT * FROM Clientes) AS Compatible''') \
    .option('user', db_user) \
    .option('password', db_psswd) \
    .option('driver', 'com.mysql.cj.jdbc.Driver') \
    .load()

df_bd.show()

+----------+--------------------+--------------+------------+--------------+----------------+-------------+-------------------+--------+
|ID_Cliente|              Nombre|ClienteFactura|ID_Categoria|ID_GrupoCompra|ID_CiudadEntrega|LimiteCredito|FechaAperturaCuenta|DiasPago|
+----------+--------------------+--------------+------------+--------------+----------------+-------------+-------------------+--------+
|         1|Tailspin Toys (He...|             1|           3|             1|           19586|         null|2013-01-01 00:00:00|       7|
|         2|Tailspin Toys (Sy...|             1|           3|             1|           33475|         null|2013-01-01 00:00:00|       7|
|         3|Tailspin Toys (Pe...|             1|           3|             1|           26483|         null|2013-01-01 00:00:00|       7|
|         4|Tailspin Toys (Me...|             1|           3|             1|           21692|         null|2013-01-01 00:00:00|       7|
|         5|Tailspin Toys (Ga...|        

Por otro lado, para persistir un dataframe en un csv se utiliza la librearía Pandas, específicamente el uso de df.toPandas().to_csv(RUTA_DESTINO) ingresando la ruta destino del nuevo archivo

El controlador que se está utilizando para conectarse a la base de datos es de tipo JDBC (Java Database Connectivity). Aunque no es necesario para este curso, si quiere saber más sobre JDBC puede consultar el siguiente recurso: 

https://www.infoworld.com/article/3388036/what-is-jdbc-introduction-to-java-database-connectivity.html


Para comprobar su comprensión de esta sección, lo invitamos a responder la pregunta:

    ¿Qué es un DataFrame en PySpark, y cómo se puede crear?

A continuación se crean dos funciones que retornan un dataframe desde una base de datos o desde un csv que le podrían ser utiles a futuro

In [10]:
def obterner_dataframe_desde_csv(_PATH, _sep):
    return spark.read.load(_PATH, format="csv", sep=_sep, inferSchema="true")

def obtener_dataframe_de_bd(db_connection_string, sql, db_user, db_psswd):
    df_bd = spark.read.format('jdbc')\
        .option('url', db_connection_string) \
        .option('dbtable', sql) \
        .option('user', db_user) \
        .option('password', db_psswd) \
        .option('driver', 'com.mysql.cj.jdbc.Driver') \
        .load()
    return df_bd

### 4.4 Select, where, filter....

Los métodos <i>select</i> y <i>where</i> funcionan de manera similar a sus homónimos en SQL. De este modo, <i>select</i> servirá para seleccionar un subconjunto de columnas y <i>where</i> permitirá filtrar filas según los valores de sus columnas. Si lo prefiere, el método <i>filter</i> funciona igual que <i>where</i>.

En el ejemplo que se muestra a continuación, se seleccionan las columnas <i>CustomerID</i>, <i>BuyingGroupID</i> y <i>CreditLimit</i>, tras lo cual se mantienen solamente aquellas filas cuyo <i>CreditLimit</i> no sea nulo y cuyo <i>CustomerID</i> sea mayor a 10.

Recuerde que tiene a su disposición los recursos nivelatorios, entre los cuales se encuentra bases de datos relacionales y SQL

In [12]:
# Consulta usando SQL
sql_results = obtener_dataframe_de_bd(db_connection_string, '(SELECT ID_Cliente,ID_GrupoCompra,LimiteCredito FROM Clientes WHERE LimiteCredito IS NOT NULL AND ID_Cliente>10) AS Compatible', db_user, db_psswd)
sql_results.show()

+----------+--------------+-------------+
|ID_Cliente|ID_GrupoCompra|LimiteCredito|
+----------+--------------+-------------+
|       801|          null|         3000|
|       802|          null|         2940|
|       803|          null|         2000|
|       804|          null|         2200|
|       805|          null|         3300|
|       806|          null|         3000|
|       807|          null|         3100|
|       808|          null|         1800|
|       809|          null|         1700|
|       810|          null|         1200|
|       811|          null|         2100|
|       812|          null|         2200|
|       813|          null|         2600|
|       814|          null|         2310|
|       815|          null|         2900|
|       816|          null|         2400|
|       817|          null|         2100|
|       818|          null|         1155|
|       819|          null|         1800|
|       820|          null|         3500|
+----------+--------------+-------

El resultado anterior se puede replicar usando select, where o filter de PySpark de la siguiente manera:

In [13]:
df_bd_small = df_bd.select('ID_Cliente','ID_GrupoCompra', 'LimiteCredito').where((df_bd['LimiteCredito'].isNotNull()) & (df_bd['ID_Cliente'] > 10))
   
df_bd_small.show()

+----------+--------------+-------------+
|ID_Cliente|ID_GrupoCompra|LimiteCredito|
+----------+--------------+-------------+
|       801|          null|         3000|
|       802|          null|         2940|
|       803|          null|         2000|
|       804|          null|         2200|
|       805|          null|         3300|
|       806|          null|         3000|
|       807|          null|         3100|
|       808|          null|         1800|
|       809|          null|         1700|
|       810|          null|         1200|
|       811|          null|         2100|
|       812|          null|         2200|
|       813|          null|         2600|
|       814|          null|         2310|
|       815|          null|         2900|
|       816|          null|         2400|
|       817|          null|         2100|
|       818|          null|         1155|
|       819|          null|         1800|
|       820|          null|         3500|
+----------+--------------+-------

Usando filter, el código sería: 

```
df_bd_small = df_bd.select('ID_Cliente','ID_GrupoCompra', 'LimiteCredito').filter(df_bd['LimiteCredito'].isNotNull()).filter(df_bd['ID_Cliente'] > 10)
```

### 4.5 User Defined Functions

Aunque PySpark ofrece muchas funcionalidades, en ocasiones necesitará realizar operaciones más complejas o personalizadas, para este tipo de operaciones existen las UDF o User Defined Functions.
Las UDF permiten aplicar funciones de Python creadas por usted mismo a las columnas de un DataFrame. En el ejemplo que se presenta a continuación, se normaliza la columna <i>CreditLimit</i> utilizando una UDF:

In [14]:
#Función para realizar una normalización min-max
def normalizar(valor, minimo, maximo):
    return (valor-minimo)/(maximo-minimo)

#Se cambia el formato de la columna Limite Credito
df_bd = df_bd.withColumn('LimiteCredito', df_bd['LimiteCredito'].cast(IntegerType()))
#Se obtiene el CreditLimit mínimo
min_cred = df_bd.agg({'LimiteCredito': 'min'}).collect()[0][0]
#Se obtiene el CreditLimit máximo
max_cred = df_bd.agg({'LimiteCredito': 'max'}).collect()[0][0]

#Se define la función normalizar como una UDF 
#Se utiliza una función Lambda internamente para inyectar min_cred y max_cred pues por defecto una UDF solo puede recibir columnas por parámetro.
norm_udf = udf(lambda x: normalizar(x, min_cred, max_cred), FloatType())

#Primero se reemplazan los valores nulos, pues la función normalizar no puede operar con valores nulos.
df_bd = df_bd.fillna({'LimiteCredito': 0.0})
df_bd = df_bd.withColumn('LimiteCredito_normalizado', norm_udf(df_bd['LimiteCredito']))

In [15]:
df_bd.select('ID_Cliente', 'LimiteCredito', 'LimiteCredito_normalizado').show()

+----------+-------------+-------------------------+
|ID_Cliente|LimiteCredito|LimiteCredito_normalizado|
+----------+-------------+-------------------------+
|         1|            0|                     null|
|         2|            0|                     null|
|         3|            0|                     null|
|         4|            0|                     null|
|         5|            0|                     null|
|         6|            0|                     null|
|         7|            0|                     null|
|         8|            0|                     null|
|         9|            0|                     null|
|        10|            0|                     null|
|        11|            0|                     null|
|        12|            0|                     null|
|        13|            0|                     null|
|        14|            0|                     null|
|        15|            0|                     null|
|        16|            0|                    

Note que primero debemos definir la función de Python, tras lo cual se puede definir la udf y finalmente, aplicar la misma sobre el DataFrame


Para comprobar su comprensión de esta sección, lo invitamos a responder las preguntas:

    ¿Qué métodos podría utilizar para reemplazar los valores nulos de una columna por el promedio de la misma?
    ¿Qué son las UDF y cómo funcionan?

## 5. Persistencia en base de datos y en csv
Para persistir los datos de un DataFrame en la base de datos es necesario indicar los datos a guardar, configurar la conexion con el servidor de base de datos, el nombre de la nueva tabla y las credenciales de autenticación. Para identificar los datos del dataframe a guardar se utilizan los métodos 'select' o 'selectExpr'. Posteriormente se configura la conexion al servidor indicando dirección ip, puerto y el nombre de la base de datos a la que se desea conectar. 

Tenga presente que la persistencia de un dataframe es independiente a la forma de con la cual se cargaron los datos (programática, con un archivo csv o desde una tabla en una base de datos). A continuación un ejemplo para el dataframe df_db

In [16]:
# Insertar en una nueva tabla
def guardar_db(df, tabla, db_connection_string, db_user, db_psswd):
    df.select('*').write.format('jdbc') \
      .mode('append') \
      .option('url', db_connection_string) \
      .option('dbtable', tabla) \
      .option('user', db_user) \
      .option('password', db_psswd) \
      .option('driver', 'com.mysql.cj.jdbc.Driver') \
      .save()

db_connection_string = 'jdbc:mysql://157.253.236.116:8080/Estudiante_43'
guardar_db(df_bd, 'ClientesProcesados', db_connection_string, db_user, db_psswd)

Por otro lado, para persistir un dataframe en un csv se utiliza la libreria pandas y el comando to_csv() ingresando la ruta destino

In [18]:
def guardar_csv(df, _PATH):
    df.toPandas().to_csv(_PATH)
    
guardar_csv(df_bd, PATH+'ClientesProcesados.csv')

## 7. Cierre

Completado este tutorial ya sabrá la forma básica de utilizar PySpark. Ya sabe cómo crear DataFrames a partir de datos existentes, cómo seleccionar columnas o filas de este Dataframe y cómo aplicar sus propias funciones a estos datos.

## 8. Información adicional

Si quiere conocer más sobre PySpark la guía más detallada es la documentación oficial, la cual puede encontrar acá: https://spark.apache.org/docs/latest/api/python/index.html <br>
Para ir directamente a la documentación de PySpark SQL, donde está la información sobre los DataFrames: https://spark.apache.org/docs/latest/api/python/pyspark.sql.html <br>

El Capítulo 2 del libro <i>Learn PySpark : Build Python-based Machine Learning and Deep Learning Models, New York: Apress. 2019</i> de Pramod Singh contiene muchos ejemplos útiles, puede encontrarlo en la biblioteca virtual de la universidad.

## 9.Preguntas frecuentes

1. En algunos casos, encontrará también información sobre <i>Pandas_UDF</i>. <i>Pandas_UDF</i> son también User Defined Functions, por lo general los Pandas UDF son más eficientes que los UDF tradicionales, sin embargo, hay un bug con la versión de PySpark y de Java que se está usando, lo que previene la utilización de Pandas_UDF.

3. Si al ejecutar la configuración de la sesión Spark le aparece el error <i>Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=local[*])</i> Reinicie el kernel y vuelva a ejecutar
